In [1]:
import string
import random
from tkinter import *
from tkinter import messagebox
import sqlite3
with sqlite3.connect("users.db") as db:
    cursor = db.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS users(Username TEXT NOT NULL,GeneratedPassword TEXT NOT NULL);")
cursor.execute("SELECT * FROM users")
db.commit()
db.close()
class GUI():
    def __init__(self, master):
        self.master=master
        self.username=StringVar()
        self.passwordlen=IntVar()
        self.generatedpassword=StringVar()
        self.n_username=StringVar()
        self.n_generatedpassword=StringVar()
        self.n_passwordlen=IntVar()
        master.title('Password Generator')
        master.geometry('700x550')
        master.config(bg='#e6f7ff')
        master.resizable(False, False)
        self.title_label=Label(master,text="PASSWORD GENERATOR",anchor=N,fg='#004d99',bg='#e6f7ff',font=('Helvetica',26,
                                                                                                         'bold'))
        self.title_label.pack(pady=20)
        self.user_frame=Frame(master, bg='#e6f7ff')
        self.user_frame.pack(pady=10)
        self.user_label=Label(self.user_frame,text="Enter User Name:",font=('Helvetica',16,'bold'),bg='#e6f7ff',
                              fg='#004d99')
        self.user_label.grid(row=0,column=0,padx=10,pady=5,sticky=W)
        self.user_entry=Entry(self.user_frame,textvariable=self.n_username,font=('Helvetica',14),bd=5,relief='ridge',
                              width=25)
        self.user_entry.grid(row=0,column=1,padx=10,pady=5)
        self.length_frame=Frame(master,bg='#e6f7ff')
        self.length_frame.pack(pady=10)
        self.length_label=Label(self.length_frame,text="Enter Password Length:",font=('Helvetica',16,'bold'),bg='#e6f7ff',
                                fg='#004d99')
        self.length_label.grid(row=0, column=0, padx=10, pady=5, sticky=W)
        self.length_entry=Entry(self.length_frame, textvariable=self.n_passwordlen,font=('Helvetica',14),bd=5,
                                relief='ridge', width=25)
        self.length_entry.grid(row=0,column=1,padx=10,pady=5)
        self.generated_password_frame=Frame(master,bg='#e6f7ff')
        self.generated_password_frame.pack(pady=10)
        self.generated_password_label=Label(self.generated_password_frame,text="Generated Password:",font=('Helvetica',16,                                                            'bold'), bg='#e6f7ff', fg='#004d99')
        self.generated_password_label.grid(row=0,column=0,padx=10,pady=5,sticky=W)
        self.generated_password_entry=Entry(self.generated_password_frame,textvariable=self.n_generatedpassword,
                                            font=('Helvetica',14),bd=5,relief='ridge',width=25,fg='#DC143C')
        self.generated_password_entry.grid(row=0,column=1,padx=10,pady=5)
        self.button_frame=Frame(master, bg='#e6f7ff')
        self.button_frame.pack(pady=20)
        button_style={
            'bd': 3,
            'relief': 'solid',
            'padx': 10,
            'pady': 10,
            'font': ('Helvetica', 14, 'bold'),
            'fg': 'white'
        }

        self.generate_button=Button(self.button_frame,text="GENERATE PASSWORD",**button_style,bg='#ff4500',
                                    command=self.generate_pass)
        self.generate_button.grid(row=0, column=0,padx=10, pady=5)
        self.accept_button = Button(self.button_frame,text="ACCEPT",**button_style,bg='#32cd32',command=self.accept_fields)
        self.accept_button.grid(row=0,column=1,padx=10,pady=5)
        self.reset_button=Button(self.button_frame, text="RESET",**button_style,bg='#1e90ff',command=self.reset_fields)
        self.reset_button.grid(row=0,column=2,padx=10,pady=5)
        self.footer_label=Label(master,text="© 2024 Password Generator Inc.",anchor=S,fg='#004d99',bg='#e6f7ff',
                                font=('Helvetica',12,'italic'))
        self.footer_label.pack(side=BOTTOM,pady=10)

    def generate_pass(self):
        upper="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        lower="abcdefghijklmnopqrstuvwxyz"
        chars="@#%&()\"?!"
        numbers="1234567890"
        upper=list(upper)
        lower=list(lower)
        chars=list(chars)
        numbers=list(numbers)
        name=self.user_entry.get()
        leng=self.length_entry.get()
        with sqlite3.connect("users.db") as db:
            cursor=db.cursor()
        if name=="":
            messagebox.showerror("Error","Name cannot be empty")
            return
        if not name.isalpha():
            messagebox.showerror("Error","Name must be a string")
            self.user_entry.delete(0, 25)
            return
        try:
            length=int(leng)
            if length < 6:
                messagebox.showerror("Error","Password must be at least 6 characters long")
                self.length_entry.delete(0, END)
                return
        except ValueError:
            messagebox.showerror("Error","Password length must be a number")
            self.length_entry.delete(0, END)
            return
        self.generated_password_entry.delete(0, END)
        u=random.randint(1,length-3)
        l=random.randint(1,length-2-u)
        c=random.randint(1,length-1-u-l)
        n=length-u-l-c
        password=random.sample(upper,u)+random.sample(lower,l)+random.sample(chars,c)+random.sample(numbers,n)
        random.shuffle(password)
        gen_passwd="".join(password)
        self.generated_password_entry.insert(0,gen_passwd)

    def accept_fields(self):
        with sqlite3.connect("users.db") as db:
            cursor=db.cursor()
            find_user=("SELECT * FROM users WHERE Username = ?")
            cursor.execute(find_user,[(self.n_username.get())])
            if cursor.fetchall():
                messagebox.showerror("Error","This username already exists!Please use another username")
            else:
                insert=("INSERT INTO users(Username, GeneratedPassword) VALUES(?, ?)")
                cursor.execute(insert, (self.n_username.get(), self.n_generatedpassword.get()))
                db.commit()
                messagebox.showinfo("Success!","Password generation successful")
                self.reset_fields()
                messagebox.showinfo("Accepted","Password accepted!")

    def reset_fields(self):
        self.user_entry.delete(0, 25)
        self.length_entry.delete(0, 25)
        self.generated_password_entry.delete(0, 25)

if __name__ == '__main__':
    root = Tk()
    pass_gen = GUI(root)
    root.mainloop()
